In [3]:
import os

import pandas              as pd
import numpy               as np
import matplotlib.pyplot   as plt

from keras.layers          import Dense
from keras.models          import Sequential, load_model
from keras.callbacks       import EarlyStopping
from keras.metrics         import categorical_crossentropy
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler

In [ ]:
# Creates folders
os.makedirs("../results/", exist_ok=True)
os.makedirs("../models/", exist_ok=True)
os.makedirs("../submission/", exist_ok=True)

In [101]:
# Importing train and test data
train = pd.read_csv('../data/train.zip')
test = pd.read_csv('../data/test.zip')

In [102]:
# Separating the target data from the train data
target = train.label
train.drop(columns= ['label'], axis = 1, inplace = True)

In [103]:
# Formatting the data
normalizer = MinMaxScaler(feature_range = (0, 1))

# Transform data in binary
lb = LabelBinarizer()

# Normalize input train and test data
x_train = normalizer.fit_transform(train)
x_test = normalizer.fit_transform(test)

y_train = lb.fit_transform(target)

In [1]:
def new_model(hidden_layers = 1, nodes = 10, patience = 5):

    file_name = f'no_cnn_{hidden_layers}_layer_{nodes}_nodes'

    # Instantiating model
    model = Sequential()

    # Creating early_stopping
    early_stopping_monitor = EarlyStopping(patience = patience,
                                           monitor='val_loss')

    # Adding hidden layers
    for _ in range (0, hidden_layers):
        
        model.add(Dense(nodes, activation = 'relu'))

    # Adding final layer
    model.add(Dense(10, activation = 'softmax'))

    # Compiling the model
    model.compile(optimizer = 'adam', 
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

    # Fit the model
    model.fit(x_train, 
              y_train, 
              epochs = 100,  
              shuffle = True, 
              verbose = 2,
              callbacks = [early_stopping_monitor],
              validation_split = 0.3)

    # Creating submission file
    submission = pd.DataFrame(np.argmax(model.predict(x_test), axis = -1), columns = ['Label'])

    # Creating ImageID index
    submission['ImageID']  = x_test.index

    submission.rename_axis('ImageId', inplace=True)

    #Saving model
    model.save(f'../models/{file_name}.h5')

    # Saving submission file
    submission.to_csv(f'../submission/{file_name}.csv')


In [2]:
# Best result so far!
new_model(hidden_layers = 3, nodes = 40)

NameError: name 'Sequential' is not defined